In [1]:
using PyPlot

In [2]:
#We need an object that has two methods: the parameters, which are its location in parameter space
#and chi2 of the model calculated from the walker's position compared to the data
type walker
    params::Array{Float64}
    chi2::Float64
end

In [3]:
#This is an object that records a given walker's position and chi2 across multiple steps 
type walker_record
    walkers::Array{walker}
    nsteps::Int64
end
walker_record(ws::Array{walker}) = walker_record(ws,length(ws))

walker_record

In [4]:
#enter each walker's record as an element of an array, one entry for each walker
type walker_array
    recs::Array{walker_record}
    nwalkers::Int8
end
walker_array(recs::Array{walker_record}) = walker_array(recs,length(recs))

walker_array

In [5]:
type model #contains a function, f(x,p), as well as nparams === length(p)
    f::Function
    nparams::Int8
end

In [6]:
#how to add a walker into a walker_record, and a walker_record into a walker_array,
#while keeping track of the number of steps taken/ the number of total walkers
import Base.push!
function push!(rec::walker_record,w::walker)
    push!(rec.walkers,w)
    rec.nsteps += 1
end
function push!(arr::walker_array,rec::walker_record)
    push!(arr.recs,rec)
    arr.nwalkers += 1
end

push! (generic function with 21 methods)

In [7]:
#returns chi2 at a position in parameter space
chi2(modl::model,params::Array{Float64},data::Array{Float64,2}) = sum(((modl.f(data[1],params)-data[2])/data[3]).^2)

chi2 (generic function with 1 method)

In [8]:
#Initialize a walker_array. nwalkers = nparams*3.0, nsteps=1, at the moment
function init_walkers(modl::model,p0::Array{Float64},p_err::Array{Float64},data::Array{Float64,2})
    nparam = modl.nparams
    nwalker = nparam*3.0
    chi_best = chi2(modl,p0,data)
    w_array = walker_array(walker_record[])
    for j=1:nwalker
        chi_trial = 1.0e100
        par_trial = p0
        while chi_trial > (chi_best + 1000)
            par_trial = p0 + p_err.*randn(nparam)
            chi_trial = chi2(modl,par_trial,data)
        end
        w = walker(par_trial,chi_trial)
        push!(w_array,walker_record([w]))
    end
    return w_array
end

init_walkers (generic function with 1 method)

In [41]:
mod_trial(x,p) = p[1].*x+p[2]
modl = model(mod_trial,2)
xdata = collect(linspace(0,1,100))
ydata = mod_trial(xdata,[1.,1.])+rand(length(xdata))/10
yerr = rand(length(xdata))/10
data = hcat(xdata,ydata,yerr)
p0 = [0.9,1.1]
perr = [0.01,0.01]
foo = init_walkers(modl,p0,perr,data)
for rec in foo.recs
    push!(rec,walker([rand(),rand()],15.0))
end
#This gives a walker_array object to test the following out on if you so choose

In [11]:
function walk(w::walker,partner::walker,ascale::Float64,modl::model,data::Array{Float64,2})
    #Takes a walker and a partner, picks a new trial step based on parameters
    z=(rand()*(sqrt(ascale)-1.0/sqrt(ascale))+1.0/sqrt(ascale))^2
    par_trial = z.*w.params.+(1.0-z).*partner.params
    # Compute chi-square:    
    chi_trial=chi2(modl,par_trial,data)
    alp = z^(modl.nparams-1)*exp(-0.5*(chi_trial - w.chi2))
    return walker(par_trial,chi_trial),alp
end

walk (generic function with 1 method)

In [31]:
function mcmc(data::Array{Float64,2},modl::model,p0::Array{Float64},p_err::Array{Float64},nstep::Int64)
    w_array = init_walkers(modl,p0,p_err,data)
    nparam = modl.nparams
    nwalker = w_array.nwalkers
    ascale = 2.0
    accept = 0
    for i=2:nstep
        #Like j = nwalkers, but now we can access the walkers directly!
        for (j,rec) in enumerate(w_array.recs)
            current_w = rec.walkers[end]
            ipartner = j
            while ipartner == j
                ipartner = ceil(Int,rand()*nwalker)
            end
            partner_w = w_array.recs[ipartner].walkers[end]
            # Now, make a trial walker
            (trial_w,alp) = walk(current_w,partner_w,ascale,modl,data)
            if alp >= rand()
            # If step is accepted, add it to the chains!
                push!(rec,trial_w)
                accept = accept + 1
            else
            # Otherwise, copy the current walker
                push!(rec,current_w)
            end
        end
        if i%1000 == 0
            frac_acc = accept/(1000*nwalker)
            println("Number of steps: $i, acceptance rate: $frac_acc")
            ascale = 1.0 + (frac_acc/0.25)*(ascale-1.0)
            accept = 0
        end
    end
    return w_array
end

mcmc (generic function with 1 method)

In [13]:
get_history(w_array::walker_array,nwalk::Int64,p1::Int64,nstart::Int64) = [w.params[p1] for w in w_array.recs[nwalk].walkers[nstart:end]]

get_history (generic function with 1 method)

In [14]:
function get_median(w_array::walker_array,p1::Int64,nstart::Int64)
    plist = Float64[]
    for i=1:w_array.nwalkers
        append!(plist,get_history(w_array,i,p1,nstart))
    end
    return median(plist)
end

get_median (generic function with 1 method)

In [15]:
function get_mean(w_array::walker_array,p1::Int64,nstart::Int64)
    plist = Float64[]
    for i=1:w_array.nwalkers
        append!(plist,get_history(w_array,i,p1,nstart))
    end
    return mean(plist)
end

get_mean (generic function with 1 method)

In [16]:
function get_std(w_array::walker_array,p1::Int64,nstart::Int64)
    plist = Float64[]
    for i=1:w_array.nwalkers
        append!(plist,get_history(w_array,i,p1,nstart))
    end
    return std(plist)
end

get_std (generic function with 1 method)

In [17]:
function get_params(w_array::walker_array)
    # Now, determine time of burn-in by calculating first time median is crossed:
    iburn = 0
    nparam = length(w_array.recs[1].walkers[1].params)
    nwalker = w_array.nwalkers
    nstep = w_array.recs[1].nsteps
    for i=1:nparam
        med_param=get_median(w_array,i,1)
        for rec in w_array.recs
            istep=2
            while (rec.walkers[istep].params[i] > med_param) == (rec.walkers[istep-1].params[i] > med_param) && (istep < nstep)
              istep=istep+1
            end
            if istep >= iburn
              iburn = istep
            end
        end
    end
    params = Float64[get_mean(w_array,i,iburn) for i=1:nparam]
    stds = Float64[get_std(w_array,i,iburn) for i=1:nparam]
    return params,stds
end

get_params (generic function with 1 method)

In [18]:
function plot_walkers(w_array::walker_array,p1::Int64,p2::Int64)
    pones = Float64[]
    ptwos = Float64[]
    for rec in w_array.recs
        push!(pones,rec.walkers[end].params[p1])
        push!(ptwos,rec.walkers[end].params[p2])
    end
    scatter(pones,ptwos)
end

plot_walkers (generic function with 1 method)

In [32]:
function run_mcmc(data::Array{Float64,2},modl::model,p0::Array{Float64},p_err::Array{Float64},nstep::Int64)
    final_walkers = mcmc(data,modl,p0,p_err,nstep)
    for i=1:modl.nparams
        for j=i:modl.nparams
            if (j!=i)&& (j>i)
                plot_walkers(final_walkers,i,j)
                show()
                clf()
            end
        end
    end
    for (par,std) in zip(get_params(final_walkers))
        println("$param +/- $std")
    end
end

run_mcmc (generic function with 1 method)